In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine
import sql  # the patched version (file is named sql.py)
from sqlalchemy import create_engine 

In [2]:

db1 = psycopg2.connect(dbname='postgres', user='postgres', 
                        password='1', host='localhost', port=5432)
db = create_engine('postgresql+psycopg2://postgres:1@localhost:5432/postgres')
sql = db1.cursor()

In [3]:
df_trans=pd.read_csv('E:\Vakansy\magnit\shedulers.csv')


In [4]:
df_trans.to_sql('T_CONTRACTOR_SHERULER', con=db, index=False, if_exists='replace')

In [5]:
sql.execute("""CREATE TABLE T_CONTRACTOR_SHERULER
(NAME varchar, SHEDULE varchar, DATE_BEGIN timestamp, DATE_END timestamp,
constraint ID_NAME primary key(NAME, DATE_BEGIN),
constraint date_check CHECK(DATE_BEGIN < DATE_END));
""")


In [6]:
sql.execute("""COPY T_CONTRACTOR_SHERULER(name, shedule, date_begin, date_end) FROM 'E:\\Vakansy\\magnit\\shedulers.csv' DELIMITER ';' CSV HEADER;""")

In [7]:
sql.execute("""CREATE TABLE T_CONTRACTOR_WORK_DAY
(ID int, NAME varchar, DATE_BEGIN timestamp, DATE_END timestamp);""")


In [8]:
sql.execute("""create or replace procedure inserter(NAME varchar, DATE_BEGIN timestamp, DATE_END timestamp)
language plpgsql    
as $$
begin
insert into T_CONTRACTOR_WORK_DAY (NAME, DATE_BEGIN, DATE_END)
(select name, 
case when d='д' then date_begin+interval '8 hour' 
when d='с' then date_begin+interval '8 hour' 
when d='н' then date_begin+interval '20 hour'
end as date_begin,
case when d='д' then date_end+interval '20 hour'
when d='с' then date_end+interval '32 hour' 
when d='н' then date_end+interval '32 hour'
end as date_end
from
(select *
from 
(select name, shedule, date_begin as date, d, row_number, date_begin+(row_number * interval'1 day') as date_begin, 
date_begin+(row_number * interval'1 day') as date_end  from 
(select name, shedule, date_begin, d, row_number () over (partition by shedule, name order by name) from 
(select name, shedule, date_begin, regexp_split_to_table(shedule, '\s*') d from T_CONTRACTOR_SHERULER) as a1) 
as a2) as a3 
where d!='в') as a4);
commit;
end;$$""")

# 4 задание


In [9]:
import records
dtb = records.Database('postgresql+psycopg2://postgres:1@localhost:5432/T_CONTRACTOR_SHERULER')

In [10]:
dtb.query("""select count(d), name
from
(select *
from 
(select name, shedule, date_begin as date, d, row_number, date_begin+(row_number * interval'1 day') as date_begin, 
date_begin+(row_number * interval'1 day') as date_end  from 
(select name, shedule, date_begin, d, row_number () over (partition by shedule, name order by name) from 
(select name, shedule, date_begin, regexp_split_to_table(shedule, '\s*') d from T_CONTRACTOR_SHERULER) as a1) 
as a2) as a3 
where d!='в') as a4
where d='д'
group by name;""").export('df')

,count,name
0,3,Поставщик 2
1,4,Поставщик 3
2,3,Поставщик 1


In [11]:
dtb.query("""select * from (select name, count(d)
from
(select *
from 
(select name, shedule, date_begin as date, d, row_number, date_begin+(row_number * interval'1 day') as date_begin, 
date_begin+(row_number * interval'1 day') as date_end  from 
(select name, shedule, date_begin, d, row_number () over (partition by shedule, name order by name) from 
(select name, shedule, date_begin, regexp_split_to_table(shedule, '\s*') d from T_CONTRACTOR_SHERULER) as a1) 
as a2) as a3 
where d!='в') as a4
where extract (month from date_begin)=1
group by name) as a5
where count>=10;""").export('df')

,name,count
0,Поставщик 1,11
1,Поставщик 2,11


In [12]:
dtb.query("""select distinct(name) from
(select *
from 
(select name, shedule, date_begin as date, d, row_number, date_begin+(row_number * interval'1 day') as date_begin, 
date_begin+(row_number * interval'1 day') as date_end  from 
(select name, shedule, date_begin, d, row_number () over (partition by shedule, name order by name) from 
(select name, shedule, date_begin, regexp_split_to_table(shedule, '\s*') d from T_CONTRACTOR_SHERULER) as a1) 
as a2) as a3 
where d!='в') as a4
where extract (month from date_begin)=1 and extract (day from date_begin) in (14, 15, 16)""").export('df')

,name
0,Поставщик 1
1,Поставщик 2
